# Notebook to download Sentinel-2 Satellite GeoTIFF images of forested areas in California before documented wildfires using Google Earth Engine (GEE)
## The data points and polygons were gathered from the folowing [Esri Living Atlas Layer ](https://pns.maps.arcgis.com/home/item.html?id=6fd0d8d6f47d414da7bcb1dcd0539999)
- The layer was preprocessed in ArcGIS Pro:
    - The layer was filtered to get dates from 2016 because we use sentinel-2 imagery.
    - The layer was clipped using the Forest Service of the Original Proclaimed National Forest Land layer.
    - The Feature to point tool was used.
    - The Get XY point tool was applied.
    - The final layer attribute table was saved as an excel file "fireForest.xls"

#### Import packages and libraries

In [1]:
#Import the necessary libraries 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyproj
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import Point, Polygon
from geopy.distance import great_circle
import ee
import geemap
import numpy as np
import geetools
import time

## Code to filter dates to: 
- Use Sentinel-2 
- Change dates outside of the the green up to months from march to september for better results

In [2]:
# Read the Excel file
excel_file_path = 'ExcelFiles/fireForest.xls'  
df = pd.read_excel(excel_file_path)


# Use the years we know Sentinel was active
mask2 = (df['ALARM_DATE'].dt.year >= 2016)
df = df[mask2]

# Parse date columns to datetime format
df['ALARM_DATE'] = pd.to_datetime(df['ALARM_DATE'], format='%d/%m/%Y', errors='coerce')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'], format='%d/%m/%Y', errors='coerce')
# Define a function to adjust dates
def adjust_dates(row):
    # Specify the updated valid range
    valid_range = range(3, 10)

    # Adjust alarm date to the closest prior date in the range
    if not pd.isna(row['ALARM_DATE']) and row['ALARM_DATE'].month not in valid_range:
        closest_prior_date = row['ALARM_DATE'] - pd.DateOffset(months=row['ALARM_DATE'].month - max(valid_range))
        row['ALARM_DATE'] = closest_prior_date

    # Adjust containment date to the closest later date in the range
    if not pd.isna(row['CONT_DATE']) and row['CONT_DATE'].month not in valid_range:
        closest_later_date = row['CONT_DATE'] + pd.DateOffset(months=min(valid_range) - row['CONT_DATE'].month)
        row['CONT_DATE'] = closest_later_date

    return row

# Apply the function to the DataFrame
df = df.apply(adjust_dates, axis=1)


## Code to change NAD83 coordinates to WGS84

In [3]:

# Define the source and target coordinate systems

source_crs = pyproj.CRS("EPSG:3310")  # NAD 1983 California Albers (Meters)
target_crs = pyproj.CRS("EPSG:4326")  # WGS 84 (EPSG:4326)

# Create a transformer for the coordinate transformation
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Iterate through rows and transform coordinates
for index, row in df.iterrows():
    x = row['POINT_X']
    y = row['POINT_Y']
    
    # Perform the coordinate transformation
    lon, lat = transformer.transform(x, y)
    
    # Update the DataFrame with transformed coordinates
    df.at[index, 'POINT_X'] = lon
    df.at[index, 'POINT_Y'] = lat

    # Create a mask to filter fires that occurred between March and September

# Save the DataFrame with transformed coordinates to a new Excel file
output_excel_file = 'ExcelFiles/FIRES.xlsx'
df.to_excel(output_excel_file, index=False)

print("Coordinate transformation complete. Output saved to:", output_excel_file)



Coordinate transformation complete. Output saved to: ExcelFiles/FIRES.xlsx


## Code to create the desired Regions Of Interests (ROI) to use in GEE
#### The ROI is created as a square

In [4]:
# Read the Excel file containing the data
df = pd.read_excel('ExcelFiles/FIRES.xlsx')

# Define the size of the square (miles)
square_side_length_miles = 15

# Create an empty DataFrame to store vertices
vertices_df = pd.DataFrame()

# Function to calculate square vertices and append them to the vertices DataFrame
def calculate_square(row):
    center_lat = row['POINT_Y']  # Latitude from 'POINT_Y'
    center_lon = row['POINT_X']  # Longitude from 'POINT_X'

    # Calculate half of the side length in kilometers
    half_side_length_km = square_side_length_miles * 1.60934  # Convert miles to kilometers

    # Calculate square vertices using great-circle distance
    top_left = great_circle(kilometers=half_side_length_km).destination((center_lat, center_lon), 45)
    top_right = great_circle(kilometers=half_side_length_km).destination((center_lat, center_lon), 135)
    bottom_left = great_circle(kilometers=half_side_length_km).destination((center_lat, center_lon), 315)
    bottom_right = great_circle(kilometers=half_side_length_km).destination((center_lat, center_lon), 225)

    # Create a dictionary with the vertices
    vertices = {
        'y1': top_left.longitude,
        'x1': top_left.latitude,
        'y2': top_right.longitude,
        'x2': top_right.latitude,
        'y3': bottom_right.longitude,
        'x3': bottom_right.latitude,
        'y4': bottom_left.longitude,
        'x4': bottom_left.latitude,
        'y5': center_lon,
        'x5': center_lat  # Include the center point as the fifth vertex
    }

    return pd.Series(vertices)

# Calculate square vertices and append them to the vertices DataFrame
vertices_df = df.apply(calculate_square, axis=1)

# Save the vertices DataFrame to a new Excel file
output_file_path = 'ExcelFiles/output_vertices.xlsx'  
vertices_df.to_excel(output_file_path, index=False, engine='openpyxl')
print(f"Vertices saved to {output_file_path}")


Vertices saved to ExcelFiles/output_vertices.xlsx


#### Code to check if the ROIs were correctly created 

In [5]:
# Read point data from FIRES.xlsx
point_df = pd.read_excel('ExcelFiles/FIRES.xlsx')

# Read ROI vertices data from output_vertices.xlsx
roi_df = pd.read_excel('ExcelFiles/output_vertices.xlsx')

print(len(roi_df), len(point_df))
# Create Shapely points from point coordinates
points = [Point(xy) for xy in zip(point_df['POINT_Y'], point_df['POINT_X'])]

# Create Shapely polygons from ROI vertices
roi_polygons = [Polygon([(row['x1'], row['y1']), (row['x2'], row['y2']), (row['x3'], row['y3']), (row['x4'], row['y4']), (row['x5'], row['y5'])]) for _, row in roi_df.iterrows()]

# Check if each point is within any of the ROI polygons
within_roi = [any(point.within(roi_polygon) for roi_polygon in roi_polygons) for point in points]

# Add a new column to the point dataframe to indicate if each point is within an ROI
point_df['Within_ROI'] = within_roi

# Save the updated point dataframe to a new Excel file
point_df.to_excel('ExcelFiles/FIRES_with_ROI_status.xlsx', index=False)
print("Point status with respect to ROI saved to ExcelFiles/FIRES_with_ROI_status.xlsx")


591 591
Point status with respect to ROI saved to ExcelFiles/FIRES_with_ROI_status.xlsx


## Code to create the dates for before and after the fire event to use in GEE

#### Define the times

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Read the Excel file
excel_file = 'ExcelFiles/FIRES.xlsx'
df = pd.read_excel(excel_file)

# Ensure columns have meaningful names
df = df.rename(columns={'ALARM_DATE': 'Incident_Date', 'CONT_DATE': 'Containment_Date'})

# Custom function to parse dates and handle invalid dates
def parse_date(date_str):
    try:
        date = pd.to_datetime(date_str)
        if date.year < 1 or date.year > 9999:
            return None  # Invalid year
        return date
    except:
        return None  # Parsing error

# Apply custom date parsing and validation
df['Incident_Date'] = df['Incident_Date'].apply(parse_date)
df['Containment_Date'] = df['Containment_Date'].apply(parse_date)

# Handle invalid dates by setting a default date (adjust as needed)
default_date = datetime(2020, 1, 1)

# Calculate the desired dates to add

df['15_Days_Before_Incident'] = df['Incident_Date'] - timedelta(days=16)
df['1_Day_Before_Incident'] = df['Incident_Date'] - timedelta(days=1)
df['1_Day_After_Containment'] = df['Containment_Date'] + timedelta(days=1)
df['15_Days_After_Containment'] = df['Containment_Date'] + timedelta(days=16)

# Calculate 1 day before the Incident Date

# Replace invalid dates with the default date
df.fillna(default_date, inplace=True)

# Extract the relevant columns
df = df[['15_Days_Before_Incident', '1_Day_Before_Incident', '1_Day_After_Containment', '15_Days_After_Containment', 'Incident_Date', 'Containment_Date']]

# Save the new DataFrame to an Excel file
new_excel_file = 'ExcelFiles/DatesFire.xlsx'
df.to_excel(new_excel_file, index=False)
print(f"The Excel file with the dates has been created: {new_excel_file}")


The Excel file with the dates has been created: ExcelFiles/DatesFire.xlsx


#### Get the dates into the format needed by GEE

In [7]:
#Import the data 

dates=pd.read_excel('ExcelFiles/DatesFire.xlsx')


# Define a function to convert the date format
def convert_date_format(date_str):
    try:
        # Parse the input date string
        parsed_date = pd.to_datetime(date_str, format='%d/%m/%Y %I:%M:%S %p')
        # Convert to the desired format 'year-month-day'
        return parsed_date.strftime('%Y-%m-%d')
    except Exception as e:
        # Handle any parsing errors gracefully
        return None

# List of columns to update
columns_to_update = ['15_Days_Before_Incident', '1_Day_Before_Incident', '1_Day_After_Containment', '15_Days_After_Containment', 'Incident_Date', 'Containment_Date']



# Apply the function to the specified columns
for column in columns_to_update:
    dates[column] = dates[column].apply(convert_date_format)


## Define and concatenate the arrays needed for GEE

In [8]:
region_data = pd.read_excel('ExcelFiles/output_vertices.xlsx')

# Read the original Excel file 'FIRES.xlsx'
original_df = pd.read_excel('ExcelFiles/FIRES.xlsx')

# Extract the last two columns
new_df = original_df[['OBJECTID','POINT_X', 'POINT_Y']]

# Save the new DataFrame to a new Excel file
output_file_path = 'ExcelFiles/Point.xlsx'
new_df.to_excel(output_file_path, index=False, engine='openpyxl')
point_data =  pd.read_excel('ExcelFiles/Point.xlsx')

# Concatenate the DataFrames along columns (axis=1)
concatenated_data = pd.concat([point_data, dates, region_data,], axis=1)
data = concatenated_data.values


In [9]:
new_excel_file = 'ExcelFiles/DataFireFinal.xlsx'
concatenated_data.to_excel(new_excel_file, index=False)

## Download the Before Wildfire RGB and NDVI images using GEE

### Install and authenticate GEE

In [10]:

#Authenticate 
ee.Authenticate()

#initialize ee
ee.Initialize()


Enter verification code:  4/1AfJohXlaWMOVkLziyDw0-eaQ0b1zzCOSVq6VOgYLoVPr_xOdIzElErUNzBI



Successfully saved authorization token.


### Export the multiband RGB and NDVI GeoTIFFs (B2, B3, B4, and B8) images for Before the fire event using GEE.

In [14]:
num = []
for i in range(0, 1000):
    num.append(i)

# Define the bands you want to download
bands_rgb = ['B4', 'B3', 'B2']  # Red, Green, Blue
bands_ndvi = ['B8', 'B4']  # NIR, Red

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

for i in num:
    # Set the center point
    point = ee.Geometry.Point(round(data[i][1], 2), round(data[i][2], 2))

    # Call an image for the selected point
    tile = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(point) \
        .filterDate(data[i][3], data[i][4]) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .first()
    # Check the properties of the image
    image_properties = tile.getInfo()
    cloudy_percentage = image_properties.get('properties', {}).get('CLOUDY_PIXEL_PERCENTAGE', 0)
    if cloudy_percentage <= 10:
        # Define the ROI
        roi = ee.Geometry.Polygon([[
            [data[i][9], data[i][10]],
            [data[i][11], data[i][12]],
            [data[i][13], data[i][14]],
            [data[i][15], data[i][16]],
            [data[i][17], data[i][18]]]])

        # Add NDVI to the selected bands
        tile_with_ndvi = calculate_ndvi(tile)

        # Select RGB and NDVI bands
        rgb_tile = tile.select(bands_rgb)
        ndvi_tile = tile_with_ndvi.select(['NDVI'])

        # Export RGB image to Google Drive
        rgb_task = ee.batch.Export.image.toDrive(**{
            'image': rgb_tile,
            'description': 'RGB_BeforeFire' + str(data[i][0]),
            'folder': 'GEE_FireImagesRGB_before',
            'scale': 10,  # Adjust the scale as needed
            'region': roi.getInfo()['coordinates'],
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
        })
        rgb_task.start()

        # Export NDVI image to Google Drive
        ndvi_task = ee.batch.Export.image.toDrive(**{
            'image': ndvi_tile,
            'description': 'NDVI_BeforeFire' + str(data[i][0]),
            'folder': 'GEE_FireImagesNDVI_before',
            'scale': 10,  # Adjust the scale as needed
            'region': roi.getInfo()['coordinates'],
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
        })
        ndvi_task.start()

        # Wait for the tasks to complete
        while not (rgb_task.status()['state'] in ['COMPLETED', 'FAILED', 'CANCELLED'] and
                   ndvi_task.status()['state'] in ['COMPLETED', 'FAILED', 'CANCELLED']):
            print('RGB Task is', rgb_task.status()['state'])
            print('NDVI Task is', ndvi_task.status()['state'])
            time.sleep(90)  # Wait for 90 seconds before checking again

        # Check the final task statuses
        rgb_task_status = rgb_task.status()
        ndvi_task_status = ndvi_task.status()
        print("RGB Task Status:", rgb_task_status)
        print("NDVI Task Status:", ndvi_task_status)
        print("RGB Task Error Message:", rgb_task_status.get("error_message", "No error message"))
        print("NDVI Task Error Message:", ndvi_task_status.get("error_message", "No error message"))
    else:
        print(f"Skipping image {i} due to cloudy percentage ({cloudy_percentage}%) > 10%")
    

RGB Task is READY
NDVI Task is READY
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task Status: {'state': 'COMPLETED', 'description': 'RGB_BeforeFire621', 'creation_timestamp_ms': 1700883210281, 'update_timestamp_ms': 1700883506721, 'start_timestamp_ms': 1700883217109, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1k4rzxlY5lh0NbxBdidO50vf8cpjLlJXf'], 'attempt': 1, 'batch_eecu_usage_seconds': 40.04901123046875, 'id': 'NE6DYBCB4DCGOKZMQJBVZQZR', 'name': 'projects/earthengine-legacy/operations/NE6DYBCB4DCGOKZMQJBVZQZR'}
NDVI Task Status: {'state': 'COMPLETED', 'description': 'NDVI_BeforeFire621', 'creation_timestamp_ms': 1700883210986, 'update_timestamp_ms': 1700883558086, 'start_timestamp_ms': 1700883298086, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1c1vcS-WWjBZ6WNSWMbW6-vaM3X95jT-_'], 'attempt': 1, 'batch_eecu_usage_seconds': 

IndexError: index 591 is out of bounds for axis 0 with size 591